# TRANSFORM TESTING
This notebook helps test transforms using local files as input

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import numpy as np
import scipy.signal

In [ ]:
from autolight.streaming.pyaudio import PyaudioSource
from autolight.transform.basic import BasicTransform

In [ ]:
src = PyaudioSource(2, 44100, dtype='float32', channels=1)
transform = BasicTransform()
audio_gen = iter(src)

In [ ]:
chunk = next(audio_gen)

In [ ]:
data = np.frombuffer(chunk, dtype=np.float32)

In [ ]:
fft = np.fft.rfft(data)

In [ ]:
def compute_binned_psd(data, nfft=1024, nbins=10):
    f,psd = signal.periodogram(x=data, nfft=nfft, return_onesided=True)
    # bin the PSD
    binned_psd = np.zeros(nbins)
    # if the bins have power of two bounds
    if 2**(nbins) == nfft:
        binned_psd[0] = psd[1]
        i = 1
        j = 2
        bin_size = 1
        while bin_size <= nfft / 4:
            for k in range(0,bin_size):
                binned_psd[i] += psd[j]
                j += 1
            i += 1
            bin_size *= 2
            
    else:
        raise ValueError('not implemented')
     
    return binned_psd

In [ ]:
psd = compute_binned_psd(data)

In [ ]:
psd

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, Bytes, List, Int

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    blocks = List(Int()).tag(sync=True) # end idx of each block
    formats = List(Unicode()).tag(sync=True)    # format of the data chunk
    data = Bytes().tag(sync=True)       # light data
    def __init__(self, data, **kwargs):
        super(HelloWidget, self).__init__(**kwargs)
        self.play(data)
    
    def prepare(self, data):
        '''
        Prepares the scene by setting the blocks and formats traitlets
        '''
        l_blocks = list()
        l_formats = list()
        
        for i,ld in enumerate(data):
            l_blocks.append(len(ld))
            
            if ld.shape[1] == 1:
                l_formates.append('w')
            elif ld.shape[1] == 3:
                l_formats.append('rgb')
            else:
                raise ValueError('invalid data shape')
            
            l_data.append(ld.tobytes())
            
        self.blocks = l_blocks
        self.formats = l_formats
        self.data = b''.join(l_data)
    
    def update(self, data):
        l_data = list()
        
        for i,ld in enumerate(data):
            l_data.append(ld.tobytes())
        
    def show(self, data):
        self.prepare(data)
        self.update(data)
        
    def play(self, data, fs=44100 / 1024): # array of numpy arrays
        self.prepare(data)
        
        for d in data:
            self.update(data)

In [ ]:
w = HelloWidget(b'\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00')

In [ ]:
%%HTML
<script src="http://localhost:8080/index.js"></script>

In [ ]:
w = HelloWidget(value=b'\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00')
w

In [ ]:
w.value = b'\x00\x00\x00\x00\x00\x00\x00\x00\x04\x01'